# Entrega laboratorio 2

## Integrantes 
-
-

In [151]:
from pyomo.environ import ConcreteModel, RangeSet, \
    Param, Var, Binary, \
    Objective, maximize, Constraint, value, \
    NonNegativeIntegers, minimize, \
    NonNegativeReals

from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from typing import List

from pyomo.opt import SolverFactory

# Punto 1

In [254]:
class AsignationProblemSolver:
    """
    A class to solve the fractional problem humanitarian mission.
    ...
    Attributes
    ----------
    """

    def __init__(self, profit, w_res, v_res, maxw_fli, maxv_fli):
        flights_num = len(profit)
        if flights_num != len(w_res):
            raise "Error: cada recurso debe tener un peso"
        elif flights_num != len(v_res):
            raise "Error: cada recurso debe tener un volumen"

        if len(maxv_fli) != len(maxw_fli):
            raise "Error: todos los aviones tienen un volumen y un peso maximo para transportar"

        self.dprofit = {i+1: profit[i] for i in range(len(profit))}
        self.dw_res = {i+1: w_res[i]*1000 for i in range(len(w_res))} # kg
        self.dv_res = {i+1: v_res[i]/self.dw_res[i+1] for i in range(len(v_res))} # m^3 / kg
        
        self.dmaxw_fli = {i+1: maxw_fli[i]*1000 for i in range(len(maxw_fli))} # kg
        self.dmaxv_fli = {i+1: maxv_fli[i] for i in range(len(maxv_fli))} # m^3

        self.num_res = len(profit)
        self.num_fli = len(maxw_fli)

    def setup(self):
        model = ConcreteModel()

        # sets
        model.R = RangeSet(1, self.num_res)
        model.F = RangeSet(1, self.num_fli)

        # params
        model.profit = Param(model.R, initialize=self.dprofit)
        model.w_res = Param(model.R, initialize=self.dw_res)
        model.v_res = Param(model.R, initialize=self.dv_res)

        model.w_fli = Param(model.F, initialize=self.dmaxw_fli)
        model.v_fli = Param(model.F, initialize=self.dmaxv_fli)

        # aux
        model.Z = Var(model.F, model.R, within=Binary)

        # decision variable
        model.X = Var(model.F, model.R, within=NonNegativeReals)

        # objective function
        def obj_function(model):
            return sum(model.X[i,j] * model.profit[j] for j in model.R for i in model.F)

        model.obj_function = Objective(rule=obj_function, sense=maximize)

        # constraints
        def volume_constr(model, i):
            return sum(model.X[i,j]*model.w_res[j] for j in model.R) <= model.v_fli[i]
        model.volume_constr = Constraint(model.F, rule=volume_constr)

        def weight_constr(model, i):
            return sum(model.X[i,j] * model.v_res[j] * model.w_res[j] for j in model.R) <= model.w_fli[i]
        model.weight_constr = Constraint(model.F, rule=weight_constr)

        def res_constr(model, j):
            return sum(model.X[i, j] for i in model.F) <= 1
        model.res_constr = Constraint(model.R, rule=res_constr)

        # def log_constr1(model, i):
        #     return model.X[i,4] >= 99 * model.Z[i,3]

        # def log_constr2(model, i):
        #     return model.X[i,3] >= 99 * model.Z[i,4]

        # model.log_constr1 = Constraint(model.F, rule=log_constr1)
        # model.log_constr2 = Constraint(model.F, rule=log_constr2)
        model.medications = Constraint(expr=lambda model: model.X[1,2] == 0)
        model.constr = Constraint(model.F, expr=lambda model, i: model.X[i,3] + model.X[i, 4] <= 1)
        model.r = Constraint(model.F, expr=lambda model, i: (model.X[i,3] / 300) == 0)

        self.model = model

    def solve(self):
        solver = SolverFactory('glpk')
        solver.solve(self.model)

    def print_solution(self):
        for i in self.model.F:
            print(f'En el avion {i} van los recursos:')
            v, w = 0, 0
            for j in self.model.R:
                print(f'\t[->] El recurso {j} peso: ')
                v += self.model.v_res[j] # kg
                w += self.model.w_res[j] # kg
            print('\t---- STATS ----')
            #print(f'\tPeso usado: {w/1000}/{self.model.w_fli[i]/1000}')
            print()

        print(f'\nProfit: {sum([self.model.X[i,j].value for j in self.model.R for i in self.model.F])}')
                

In [255]:
profit = [50, 100, 120, 60, 40]
w_res = [15, 5, 20, 18, 10]
v_res = [8, 2, 10, 12, 6]

maxw_fli = [30, 40, 50]
maxv_fli = [25, 30, 35]

solver = AsignationProblemSolver(profit, w_res, v_res, maxw_fli, maxv_fli)
solver.setup()
solver.solve()
solver.print_solution()

En el avion 1 van los recursos:
	[->] El recurso 1 peso: 
	[->] El recurso 2 peso: 
	[->] El recurso 3 peso: 
	[->] El recurso 4 peso: 
	[->] El recurso 5 peso: 
	---- STATS ----

En el avion 2 van los recursos:
	[->] El recurso 1 peso: 
	[->] El recurso 2 peso: 
	[->] El recurso 3 peso: 
	[->] El recurso 4 peso: 
	[->] El recurso 5 peso: 
	---- STATS ----

En el avion 3 van los recursos:
	[->] El recurso 1 peso: 
	[->] El recurso 2 peso: 
	[->] El recurso 3 peso: 
	[->] El recurso 4 peso: 
	[->] El recurso 5 peso: 
	---- STATS ----


Profit: 0.015500000000000002


# Punto 2

In [147]:
class TransportProblemSolver:
    """
    A class to solve the transport problem.
    ...
    Attributes
    ----------
    costs : List[List[float]]
        A matrix in which each entry represents the cost of going from a distribution
        city to a destination city.
    offer : List[int]
        An array with the availability of resources in each distribution city.
    demand : List[int]
        An array with the demand of resources in each destination city.
    """

    def __init__(self, costs: List[List[float]], offer: List[int], demand: List[int]):
        if len(costs) == 0:
            raise "Error: la matriz de costos no puede ser vacia"
        elif len(demand) != len(costs):
            raise "Error: cada j ciudad destino debe tener una fila en la matriz"
        elif len(offer) != len(costs[0]):
            raise "Error: cada i ciudad distribuidora debe tener una columna en la matriz"

        self.offer = offer
        self.costs = costs
        self.demand = demand

    def setup(self):
        model = ConcreteModel()

        # sets
        model.D = RangeSet(1, len(self.demand)) # destination
        model.O = RangeSet(1, len(self.offer)) # origin

        # params
        dict_ofert = {i+1: self.offer[i] for i in range(len(self.offer))}
        dict_demand = {i+1: self.demand[i] for i in range(len(self.demand))}
        dict_costs = {
            (i+1, j+1): self.costs[i][j]
            for j in range(len(self.costs[0])) for i in range(len(self.costs))
        }

        model.offer = Param(model.O, initialize=dict_ofert)
        model.demand = Param(model.D, initialize=dict_demand)
        model.costs = Param(model.D, model.O, initialize=dict_costs)
        
        # dec. variable
        model.X = Var(model.D, model.O, within=NonNegativeIntegers)

        # objective function
        def obj_function(model):
            return sum(model.X[i, j] * model.costs[i, j] for j in model.O for i in model.D)
        model.obj_function = Objective(rule=obj_function, sense=minimize)

        # constraints
        def offer_constr(model, j):
            return sum(model.X[i,j] for i in model.D) <= model.offer[j]
        model.offer_contr = Constraint(model.O, rule=offer_constr)

        def demand_constr(model, i):
            return sum(model.X[i,j] for j in model.O) >= model.demand[i]
        model.demand_constr = Constraint(model.D, rule=demand_constr)

        self.model = model

    def solve(self):
        solver = SolverFactory('glpk')
        solver.solve(self.model)

    def print_solution(self):
        for j in self.model.O:
            print(f"Desde {j} ciudad se ha sumistrado recursos hacia: ")
            ofert = 0
            for i in self.model.D:
                if self.model.X[i, j].value != 0:
                    ofert += self.model.X[i,j].value
                    resources = self.model.X[i,j].value
                    print(f"\t[->] destino {i} un total de {resources} recursos (min: {self.model.demand[i]})")
            print('\t---- STATS ----')
            print(f'\tused: {ofert}/{self.model.offer[j]} from distribution city\n')

In [145]:
offer = [550, 700]
costs = [
    [9999, 2.5],
    [2.5, 9999],
    [1.6, 2.0],
    [1.5, 1.0],
    [0.8, 1.0],
    [1.4, 0.8]
]
demand = [125, 175, 225, 250, 225, 200]

solver = TransportProblemSolver(costs, offer, demand)
solver.setup()
solver.solve()
solver.print_solution()

Desde 1 ciudad se ha sumistrado recursos hacia: 
	[->] destino 2 un total de 175.0 recursos (min: 175)
	[->] destino 3 un total de 225.0 recursos (min: 225)
	[->] destino 5 un total de 150.0 recursos (min: 225)
	---- STATS ----
	used: 550.0/550 from distribution city

Desde 2 ciudad se ha sumistrado recursos hacia: 
	[->] destino 1 un total de 125.0 recursos (min: 125)
	[->] destino 4 un total de 250.0 recursos (min: 250)
	[->] destino 5 un total de 75.0 recursos (min: 225)
	[->] destino 6 un total de 200.0 recursos (min: 200)
	---- STATS ----
	used: 650.0/700 from distribution city



# Punto 3

# Punto 4

# Punto 5